In [1]:
import pytraj as pt, nglview as nv

In [2]:
# !wget http://ambermd.org/tutorials/advanced/tutorial16/include/DOPC_128.inpcrd
# !wget http://ambermd.org/tutorials/advanced/tutorial16/include/DOPC_128.prmtop
traj = pt.load('DOPC_128.inpcrd', 'DOPC_128.prmtop')

In [3]:
view = nv.show_pytraj(traj)
view.clear_representations()
view

In [4]:
view.clear_representations()
view.add_line('not water and not hydrogen')

In [5]:
traj

pytraj.Trajectory, 1 frames: 
Size: 0.000713 (GB)
<Topology: 31882 atoms, 5130 residues, 4874 mols, PBC with box type = ortho>
           

In [6]:
from time import time
xyz = traj[0].xyz.astype('f4')

# Send bytes

In [40]:
mytime = time()*1000

binary = xyz.tobytes()

data_meta = {'0': dict(data=0, dtype='f4', shape=xyz.shape)}

d = dict(type='binary_single', mytime=mytime, data=data_meta)
view.send(d, buffers=[binary])

# Send base64

In [70]:
mytime = time()*1000

data_base64 = {'0': dict(data=nv.encode_numpy(xyz, dtype='f4'), 
                  dtype='f4',
                  shape=xyz.shape)}

d = dict(type='base64_single', mytime2=mytime, data=data_base64)

view.send(d)

# make sure we send the same array

In [ ]:
from pytraj.testing import aa_eq

xyz_from_bytes = np.frombuffer(binary, dtype='f4').reshape(xyz.shape)
xyz_from_base64 = nv.decode_base64(data_base64['0']['data'], shape=xyz.shape,
                                  dtype='f4')

aa_eq(xyz_from_bytes, xyz_from_base64)